# Nome do Processo - com tamanho nivel 1
<b>Author:</b> Alex Mercedes
<br><b>Creation:</b> YYYY-MM-DD
<br><b>Description:</b> Breve descrição do Processo Geral
<br><b>Pipeline Associado:</b> Endereço do pipeline asasociado dentro do Azure
<br><b>Notebook Associado:</b> Endereço do notebook asasociado dentro do Databricks

## Definição Variáveis
<b>Description:</b> Definição de variáveis e Bibliotecas utilizadas no processo

### Definição Bibliotecas utilizadas no processo

In [ ]:
# Variáveis Globais Notebook
# Import de Delta no Databricks e classes necessárias Databricks
# from databricks.sdk.dbutils import dbutils
# from databricks.sdk.runtime import spark

# from pyspark.sql import DataFrame
from delta.tables import DeltaTable
import calendar, delta

# Importação de funções de data (from datetime import *)
from datetime import datetime, date, timedelta

# Importação de funções de sql funções (from pyspark.sql.functions import *)
from pyspark.sql.functions import row_number, when, expr, col, lit, sum, regexp_replace, upper, trim, coalesce, add_months, to_date, to_timestamp, concat, aggregate, last_day, date_trunc, max, min, cast, first, greatest, make_date

# Importação de funções de sql datatypes (from pyspark.sql.types import *)
from pyspark.sql.types import IntegerType, StringType, DecimalType, TimestampType, DateType, StructType, StructField

# Importação de funções de sql window (from pyspark.sql.window import *)
from pyspark.sql.window import Window

### Definição Variaveis e Schemas a serem montados utilizadas no processo

In [ ]:
# Variáveis de controle
var_etapa = "0 - Variaveis"
var_schema = "overwriteSchema"
var_modeatualiza = "append"

# Parâmetros de execução
var_dt_ini = date.today()
var_dt_fim = date.today() - timedelta(days=1)
var_dt_referencia = "202511"
var_sku = ['000000000']

# Camadas
bronze = "15_curr"
silver = "20_edm"
gold = "30_dtmart"

# Parâmetros Notebook (Quando necessário passar/retornar um valor no notebook)
# dbutils.widgets.text("val_dtexec_ntbk", "0")
# val_dtexec_calc = dbutils.widgets.get("val_dtexec_ntbk")
val_lnh = 0

# Sources envolvidos
tb_source_1 = "dim_nome_source1"
tb_source_2 = "dim_nome_source2"
# tb_source_3 = "dim_nome_source3"

# Targets envolvidos
tb_target_1 = "dim_target1"
tb_target_coment_1 = "Descrição do comentário da Dimensão a ser gravado na tabela"

tb_target_2 = "dim_target2"
tb_target_coment_2 = "Descrição do comentário da Dimensão a ser gravado na tabela"

# Schemas envolvidos
var_schema_struct_nome_1 = StructType([
                                        StructField('id_key', StringType(), True),
                                        StructField('id_material', StringType(), True),
                                        StructField('id_uf', StringType(), True),
                                        StructField('dsc_cnl', StringType(), True),
                                        StructField('dt_mes', StringType(), True),
                                        StructField('dsc_scenario', StringType(), True),
                                        # StructField('', StringType(), True),
                                    ])

var_schema_struct_nome_2 = StructType([
                                        StructField('id_material', StringType(), True),
                                        # StructField('', StringType(), True),
                                        StructField('id_uf', StringType(), True),
                                        StructField('dsc_cnl', StringType(), True),
                                        StructField('dt_mes', StringType(), True),
                                        StructField('qtd_vol_baseline', StringType(), True),
                                    ])

## Obter DataFrames
<b>Description:</b> Definição de Dataframes utilizadas no processo<br>
<b>Dataframe_1:</b> df_dataframe_1<br>
<b>Regras_1:</b><br>
- Definição de regra do DataFrame
- Descrição de regra de algum campo<br>

<b>Dataframe_2:</b> df_dataframe_2<br>
<b>Regras_2:</b><br>
- Definição de regra do DataFrame
- Descrição de regra de algum campo<br>

In [ ]:
try:
# Identificar existência da tabela no BD
  var_etapa = "1 - Descrição da etapa 1 identificando a tabela de entrada 1 com algum eventual erro."
  if spark.catalog.tableExists(f"{silver}.{tb_source_1}"):
    var_tb_source_1 = spark.sql(f"describe history {gold}.{tb_source_1}") \
                            .limit(1) \
                            .collect()[0][0]
    print(var_etapa," (",var_tb_source_1,")")

  var_etapa = "2 - Descrição da etapa 2 identificando a tabela de entrada 2 com algum eventual erro."
  if spark.catalog.tableExists(f"{silver}.{tb_source_2}"):
    var_tb_source_2 = spark.sql(f"describe history {silver}.{tb_source_2}") \
                            .limit(1) \
                            .collect()[0][0]
    print(var_etapa," (",var_tb_source_2,")")

  var_etapa = "3 - Descrição da etapa 2 identificando a tabela de saída 1 com algum eventual erro."
  if spark.catalog.tableExists(f"{gold}.{tb_target_1}"):
    var_tb_target_1 = spark.sql(f"describe history {gold}.{tb_target_1}") \
                                          .limit(1) \
                                          .collect()[0][0]
    print(var_etapa," (",var_tb_target_1,")")
    var_modeatualiza_1 = "append"
  else:
    var_tb_target_1 = 0
    var_modeatualiza_1 = "overwrite"

# Comentário do Dataframe com o select da tabela source envolvida
  var_etapa = "4 - Obtendo Dados da tabela XXXXXXX considerando os filtros de XXXXX."
  df_nome_tb1 = spark.read\
                      .table(f"{silver}.{tb_source_1}")\
                      .select(
                                col("id_material")
                                , col("id_uf")
                                , col("dsc_cnl")
                                , col("dt_mes")
                                , col("dsc_scenario")
                                , regexp_replace(col("qtd_volume"), ',', '.').try_cast("double").alias("qtd_volume")
                                , col("dsc_versao")
                              )\
                        .where(
                                (lit('a') == lit('a'))
                                & (col("dsc_versao") == val_dtexec_calc)
                              )\

  # Tabela Materiais (heiaepbr001dwe01.30_dtmart.dim_hierarquia_produto_refact).
  var_etapa = "5 - Obtendo Dados da tabela XXXXXXX considerando os filtros de XXXXX"
  df_nome_tb2 = spark.read\
                      .table(f"{gold}.{tb_source_3}")\
                      .select(
                              col("id_material")
                              , col("dsc_material")
                              , col("id_submarca")
                              , col("id_pacote")
                            )\

except Exception as e:
  print("Erro - ", var_etapa, f" Erro : {e}")

## Transformação dos dados nos Dataframes (Tratamento e Regras)
<b>Description:</b> Fase de tratamentos de Dados e Calculos dos dataframes<br>

### Tratamentos de campos, Agrupamentos, Filtragens etc
<b>Description:</b> Transformação do dataframe <br>
<b>Regras:</b><br>
- A coluna **"Revenue"** representa o calculo das colunas/campos 
    - SOMA(Gross_Revenue + Sales_Tax + Tax_Incentives + Discounts_to_Customers + Variable_Selling_Expenses)

In [ ]:
try:
# if 1 == 1:
# Calculo e identificações de Key_X por possibilidades de agrupamentos.
  var_etapa = "8 - Verificando possibilidades de agrupamentos."
  df_base_reais_chaves = df_base_reais.select(
                                                when((
                                                        col("dsc_fbrc_prd").isNotNull()
                                                        & col("dsc_fbrc_vnd").isNotNull()
                                                        & col("id_uf").isNotNull()
                                                        & col("dsc_cnl").isNotNull()
                                                        & col("dt_mes").isNotNull()
                                                      ), lit('Key_A'))\
                                                .otherwise(when((
                                                                  col("dsc_fbrc_prd").isNull()
                                                                  & col("dsc_fbrc_vnd").isNotNull()
                                                                  & col("id_uf").isNotNull()
                                                                  & col("dsc_cnl").isNotNull()
                                                                  & col("dt_mes").isNotNull()
                                                                ), lit('Key_B'))\
                                                          .otherwise(when((
                                                                            col("dsc_fbrc_prd").isNull()
                                                                            & col("dsc_fbrc_vnd").isNull()
                                                                            & col("id_uf").isNotNull()
                                                                            & col("dsc_cnl").isNotNull()
                                                                            & col("dt_mes").isNotNull()
                                                                          ), lit('Key_C'))\
                                                                    .otherwise(when((
                                                                                      col("dsc_fbrc_prd").isNull()
                                                                                      & col("dsc_fbrc_vnd").isNull()
                                                                                      & col("id_uf").isNull()
                                                                                      & col("dsc_cnl").isNotNull()
                                                                                      & col("dt_mes").isNotNull()
                                                                                    ), lit('Key_D'))\
                                                                              .otherwise(when((
                                                                                                col("dsc_fbrc_prd").isNotNull()
                                                                                                & col("dsc_fbrc_vnd").isNull()
                                                                                                & col("id_uf").isNull()
                                                                                                & col("dsc_cnl").isNull()
                                                                                                & col("dt_mes").isNotNull()
                                                                                              ), lit('Key_E'))\
                                                                                        .otherwise(when((
                                                                                                          col("dsc_fbrc_prd").isNull()
                                                                                                          & col("dsc_fbrc_vnd").isNull()
                                                                                                          & col("id_uf").isNull()
                                                                                                          & col("dsc_cnl").isNull()
                                                                                                          & col("dt_mes").isNotNull()
                                                                                                        ), lit('Key_F'))\
                                                                                                  .otherwise(when((
                                                                                                                    col("dsc_fbrc_prd").isNull()
                                                                                                                    & col("dsc_fbrc_vnd").isNull()
                                                                                                                    & col("id_uf").isNull()
                                                                                                                    & col("dsc_cnl").isNull()
                                                                                                                    & col("dt_mes").isNull()
                                                                                                                  ), lit('Key_G'))
                                                                                                            .otherwise(lit('Key_Indefinida'))
                                                                                                            )
                                                                                                  )
                                                                                        )
                                                                              )
                                                                    )
                                                          )\
                                                .alias("id_key")
                                                , col("id_material")
                                                , col("dsc_fbrc_vnd")
                                                , col("dsc_fbrc_prd")
                                                , col("id_uf")
                                                , col("dsc_cnl")
                                                , col("dt_mes")
                                                , col("qtd_volume")
                                                , col("val_gross_revenue")
                                                , col("val_sales_tax")
                                                , col("val_tax_incentives")
                                                , col("val_discount_customers")
                                                , col("val_variable_sell_expense")
                                                , col("val_proceed_service")
                                                , col("val_product_bought")
                                                , col("val_energy_water")
                                                , col("val_packaging_materials")
                                                , col("val_raw_materials")
                                                , col("val_other_variable_expenses")
                                                , col("val_transfer_freight")
                                                , col("val_cif_freight")
                                                , col("val_cif_others")
                                              )\
                                      .groupBy(
                                                  col("id_key")
                                                  , col("id_material")
                                                  , coalesce(col("dsc_fbrc_vnd"), lit('')).alias("dsc_fbrc_vnd")
                                                  , coalesce(col("dsc_fbrc_prd"), lit('')).alias("dsc_fbrc_prd")
                                                  , coalesce(col("id_uf"), lit('')).alias("id_uf")
                                                  , coalesce(col("dsc_cnl"), lit('')).alias("dsc_cnl")
                                                  , coalesce(col("dt_mes"), lit('')).alias("dt_mes")
                                                )\
                                      .agg(
                                            sum(col("qtd_volume")).alias("qtd_volume")
  # sum(cast(replace(replace(Gross_Revenue,',',''),' ','') as DECIMAL(10,2))) / sum(cast(replace(replace(Volume,' ',''),',','') as DECIMAL(10,2))) as GR_hl,
                                            , (sum(col("val_gross_revenue")) / sum(col("qtd_volume"))).alias("val_gr_hl")
  # sum(cast(replace(replace(Sales_Tax,',',''),' ','') as DECIMAL(10,2))) / sum(cast(replace(replace(Volume,' ',''),',','') as DECIMAL(10,2))) as Duties_hl,
                                            , (sum(col("val_sales_tax")) / sum(col("qtd_volume"))).alias("val_duties_hl")
  # sum(cast(replace(replace(Tax_Incentives,',',''),' ','') as DECIMAL(10,2))) / sum(cast(replace(replace(Volume,' ',''),',','') as DECIMAL(10,2))) as Incent_hl,
                                            , (sum(col("val_tax_incentives")) / sum(col("qtd_volume"))).alias("val_incent_hl")
  # sum(cast(replace(replace(Discounts_to_Customers,',',''),' ','') as DECIMAL(10,2))) / sum(cast(replace(replace(Volume,' ',''),',','') as DECIMAL(10,2))) as Disc_hl,
                                            , (sum(col("val_discount_customers")) / sum(col("qtd_volume"))).alias("val_disc_hl")
  # sum(cast(replace(replace(Variable_Selling_Expenses,',',''),' ','') as DECIMAL(10,2))) / sum(cast(replace(replace(Volume,' ',''),',','') as DECIMAL(10,2))) as var_sell_exp_hl,
                                            , (sum(col("val_variable_sell_expense")) / sum(col("qtd_volume"))).alias("val_var_sell_exp_hl")
  # sum(cast(replace(replace(Proceeds_from_Services,',',''),' ','') as DECIMAL(10,2))) / sum(cast(replace(replace(Volume,' ',''),',','') as DECIMAL(10,2))) as proceeds_hl,
                                            , (sum(col("val_proceed_service")) / sum(col("qtd_volume"))).alias("val_proceeds_hl")
  # sum(cast(replace(replace(Revenue,',',''),' ','') as DECIMAL(10,2))) / sum(cast(replace(replace(Volume,' ',''),',','') as DECIMAL(10,2))) as rev_hl,
                                            , (sum((col("val_gross_revenue") + col("val_sales_tax") + col("val_tax_incentives") + col("val_discount_customers") + col("val_variable_sell_expense"))) / sum(col("qtd_volume"))).alias("val_rev_hl")
  # sum(cast(replace(replace(Products_bought_in_for_resale,',',''),' ','') as DECIMAL(10,2))) / sum(cast(replace(replace(Volume,' ',''),',','') as DECIMAL(10,2))) as resale_hl,
                                            , (sum(col("val_product_bought")) / sum(col("qtd_volume"))).alias("val_resale_hl")
  # sum(cast(replace(replace(Energy_Water,',',''),' ','') as DECIMAL(10,2))) / sum(cast(replace(replace(Volume,' ',''),',','') as DECIMAL(10,2))) as energy_hl,
                                            , (sum(col("val_energy_water")) / sum(col("qtd_volume"))).alias("val_energy_hl")
  # sum(cast(replace(replace(Packaging_Materials,',',''),' ','') as DECIMAL(10,2))) / sum(cast(replace(replace(Volume,' ',''),',','') as DECIMAL(10,2))) as pack_hl,
                                            , (sum(col("val_packaging_materials")) / sum(col("qtd_volume"))).alias("val_pack_hl")
  # sum(cast(replace(replace(Raw_Materials,',',''),' ','') as DECIMAL(10,2))) / sum(cast(replace(replace(Volume,' ',''),',','') as DECIMAL(10,2))) as raw_hl,
                                            , (sum(col("val_raw_materials")) / sum(col("qtd_volume"))).alias("val_raw_hl")
  # sum(cast(replace(replace(Other_Variable_Expenses,',',''),' ','') as DECIMAL(10,2))) / sum(cast(replace(replace(Volume,' ',''),',','') as DECIMAL(10,2))) as others_hl,
                                            , (sum(col("val_other_variable_expenses")) / sum(col("qtd_volume"))).alias("val_others_hl")
  # sum(cast(replace(replace(Variable_COGS,',',''),' ','') as DECIMAL(10,2))) / sum(cast(replace(replace(Volume,' ',''),',','') as DECIMAL(10,2))) as var_COGS_hl,
                                            , (sum((col("val_packaging_materials") + col("val_raw_materials") + col("val_other_variable_expenses") + col("val_proceed_service"))) / sum(col("qtd_volume"))).alias("val_cogs_hl")
  # sum(cast(replace(replace(Transfer_Freight,',',''),' ','') as DECIMAL(10,2))) / sum(cast(replace(replace(Volume,' ',''),',','') as DECIMAL(10,2))) as Transfer_hl,
                                            , (sum(col("val_transfer_freight")) / sum(col("qtd_volume"))).alias("val_transfer_hl")
  # sum(cast(replace(replace(CIF_Freight,',',''),' ','') as DECIMAL(10,2))) / sum(cast(replace(replace(Volume,' ',''),',','') as DECIMAL(10,2))) as CIF_Freight_hl,
                                            , (sum(col("val_cif_freight")) / sum(col("qtd_volume"))).alias("val_freight_hl")
  # sum(cast(replace(replace(Cif_Others,',',''),' ','') as DECIMAL(10,2))) / sum(cast(replace(replace(Volume,' ',''),',','') as DECIMAL(10,2))) as Cif_Others_hl,
                                            , (sum(col("val_cif_others")) / sum(col("qtd_volume"))).alias("val_cif_others_hl")
  # sum(cast(replace(replace(Primary_Variable,',',''),' ','') as DECIMAL(10,2))) / sum(cast(replace(replace(Volume,' ',''),',','') as DECIMAL(10,2))) as Prim_var_hl,
                                            , (sum(col("val_transfer_freight") + col("val_cif_freight") + col("val_cif_others")) / sum(col("qtd_volume"))).alias("val_prim_var_hl")
  # sum(cast(replace(replace(FGP,',',''),' ','') as DECIMAL(10,2))) / sum(cast(replace(replace(Volume,' ',''),',','') as DECIMAL(10,2))) as Variable_Margin_hl 
                                            , (sum(((col("val_gross_revenue") + col("val_sales_tax") + col("val_tax_incentives") + col("val_discount_customers") + col("val_variable_sell_expense")) + (col("val_packaging_materials") + col("val_raw_materials") + col("val_other_variable_expenses") + col("val_proceed_service")) + (col("val_transfer_freight") + col("val_cif_freight") + col("val_cif_others")))) / sum(col("qtd_volume"))).alias("val_variable_margin_hl")
                                          )\
                                      # .display()

except Exception as e:
  print("Erro - ", var_etapa, f" Erro : {e}")

### Calculos Finance
<b>Description:</b> Definição geral referente aos cálculos do processo<br>

####
<b>Description:</b> Geração do dataframe df_YYYYY com os cálculos, considerando as possibilidades de chaves informadas<br>
<b>Regras:</b><br>
- SOMA(Gross_Revenue + Sales_Tax + Tax_Incentives + Discounts_to_Customers + Variable_Selling_Expenses)


In [ ]:
try:
# if 1 == 1:
# Calculo e do estudo informado com o cruzamento da  Key_X identificada
  var_etapa = "9 - Identificando o cruzamento da possibilidades do Key_x."
  df_saida_base_estudo = df_estudo_sop_carga.alias("a")\
                                            .join(
                                                  df_base_reais_chaves.alias("b") 
                                                  , (
                                                      (
                                                        (col("a.id_material") == col("b.id_material"))
                                                        & (coalesce(col("a.dsc_fbrc_vnd"), lit("")) == coalesce(col("b.dsc_fbrc_vnd"), lit("")))
                                                        & (coalesce(col("a.dsc_fbrc_prd"), lit("")) == coalesce(col("b.dsc_fbrc_prd"), lit("")))
                                                        & (coalesce(col("a.id_uf"), lit("")) == coalesce(col("b.id_uf"), lit("")))
                                                        & (coalesce(col("a.dsc_cnl"), lit("")) == coalesce(col("b.dsc_cnl"), lit("")))
                                                        & (coalesce(col("a.dt_mes"), lit("")) == coalesce(col("b.dt_mes"), lit("")))
                                                        & (col("b.id_key") == 'Key_A')
                                                      )
                                                      |
                                                      (
                                                        (col("a.id_material") == col("b.id_material"))
                                                        & (coalesce(col("a.dsc_fbrc_vnd"), lit("")) == coalesce(col("b.dsc_fbrc_vnd"), lit("")))
                                                        & (coalesce(col("a.id_uf"), lit("")) == coalesce(col("b.id_uf"), lit("")))
                                                        & (coalesce(col("a.dsc_cnl"), lit("")) == coalesce(col("b.dsc_cnl"), lit("")))
                                                        & (coalesce(col("a.dt_mes"), lit("")) == coalesce(col("b.dt_mes"), lit("")))
                                                        & (col("b.id_key") == 'Key_B')
                                                      )
                                                      |
                                                      (
                                                        (col("a.id_material") == col("b.id_material"))
                                                        & (coalesce(col("a.id_uf"), lit("")) == coalesce(col("b.id_uf"), lit("")))
                                                        & (coalesce(col("a.dsc_cnl"), lit("")) == coalesce(col("b.dsc_cnl"), lit("")))
                                                        & (coalesce(col("a.dt_mes"), lit("")) == coalesce(col("b.dt_mes"), lit("")))
                                                        & (col("b.id_key") == 'Key_C')
                                                      )
                                                      |
                                                      (
                                                        (col("a.id_material") == col("b.id_material"))
                                                        & (coalesce(col("a.dsc_cnl"), lit("")) == coalesce(col("b.dsc_cnl"), lit("")))
                                                        & (coalesce(col("a.dt_mes"), lit("")) == coalesce(col("b.dt_mes"), lit("")))
                                                        & (col("b.id_key") == 'Key_D')
                                                      )
                                                      |
                                                      (
                                                        (col("a.id_material") == col("b.id_material"))
                                                        & (coalesce(col("a.dsc_fbrc_vnd"), lit("")) == coalesce(col("b.dsc_fbrc_vnd"), lit("")))
                                                        & (coalesce(col("a.dt_mes"), lit("")) == coalesce(col("b.dt_mes"), lit("")))
                                                        & (col("b.id_key") == 'Key_E')
                                                      )
                                                      |
                                                      (
                                                        (col("a.id_material") == col("b.id_material"))
                                                        & (coalesce(col("a.dt_mes"), lit("")) == coalesce(col("b.dt_mes"), lit("")))
                                                        & (col("b.id_key") == 'Key_F')
                                                      )
                                                      |
                                                      (
                                                        (col("a.id_material") == col("b.id_material"))
                                                        & (col("b.id_key") == 'Key_G')
                                                      )
                                                    )
                                                  , "inner"
                                                  )\
                                            .select(
                                                      col("a.*")
                                                      , col("b.*")
                                                    )\
                                            .groupBy(
                                                      col("b.id_key")
                                                      , col("a.id_material")
                                                      , coalesce(col("a.dsc_fbrc_vnd"), lit('')).alias("dsc_fbrc_vnd")
                                                      , coalesce(col("a.dsc_fbrc_prd"), lit('')).alias("dsc_fbrc_prd")
                                                      , coalesce(col("a.id_uf"), lit('')).alias("id_uf")
                                                      , coalesce(col("a.dsc_cnl"), lit('')).alias("dsc_cnl")
                                                      , coalesce(col("a.dt_mes"), lit('')).alias("dt_mes")
                                                      , col("dsc_scenario").alias("dsc_scenario")
                                                      , col("a.dsc_versao").alias("dsc_versao")
                                                    )\
                                            .agg(
                      # Estudo_SOPX	Volume,	sum(o.Volume) as Tot_Volume, 
                                                  sum(col("a.qtd_volume")).alias("tot_volume")
                      # Join_Estudo_Base	Variable_Margin,	sum(o.Volume)* r.FGP_hl as Tot_FGP,
                                                  , (sum(col("a.qtd_volume")) * sum(col("b.val_variable_margin_hl"))).alias("tot_fgp")
                      # Join_Estudo_Base	Gross_Revenue,	sum(o.Volume)* r.GR_hl as Tot_Gross_Revenue,
                                                  , (sum(col("a.qtd_volume")) * sum(col("b.val_gr_hl"))).alias("tot_gross_revenue")
                      # Join_Estudo_Base	Sales_Tax,	sum(o.Volume)* r.Duties_hl as Tot_Duties_hl,
                                                  , (sum(col("a.qtd_volume")) * sum(col("b.val_duties_hl"))).alias("tot_duties_hl")
                      # Join_Estudo_Base	Tax_Incentives,	sum(o.Volume)* r.Incent_hl as Tot_Incent_hl,
                                                  , (sum(col("a.qtd_volume")) * sum(col("b.val_incent_hl"))).alias("tot_incent_hl")
                      # Join_Estudo_Base	Discounts_to_Customers,	sum(o.Volume)* r.Disc_hl as Tot_Disc_hl,
                                                  , (sum(col("a.qtd_volume")) * sum(col("b.val_disc_hl"))).alias("tot_disc_hl")
                      # Join_Estudo_Base	Variable_Selling_Expenses,	sum(o.Volume)* r.var_sell_exp_hl as Tot_var_sell_exp_hl,
                                                  , (sum(col("a.qtd_volume")) * sum(col("b.val_var_sell_exp_hl"))).alias("tot_var_sel_exp_hl")
                      # Join_Estudo_Base	Proceeds_from_Services,	sum(o.Volume)* r.proceeds_hl as Tot_proceeds_hl,
                                                  , (sum(col("a.qtd_volume")) * sum(col("b.val_proceeds_hl"))).alias("tot_proceeds_hl")
                      # Join_Estudo_Base	Revenue,	sum(o.Volume)* r.rev_hl as Tot_rev_hl,
                                                  , (sum(col("a.qtd_volume")) * sum(col("b.val_rev_hl"))).alias("tot_rev_hl")
                      # Join_Estudo_Base	HUB_FM,	sum(o.Volume)* r.resale_hl as Tot_resale_hl,
                                                  , (sum(col("a.qtd_volume")) * sum(col("b.val_resale_hl"))).alias("tot_resale_hl")
                      # Join_Estudo_Base	Energy_Water,	sum(o.Volume)* r.energy_hl as Tot_energy_hl,
                                                  , (sum(col("a.qtd_volume")) * sum(col("b.val_energy_hl"))).alias("tot_energy_hl")
                      # Join_Estudo_Base	Packaging_Materials,	sum(o.Volume)* r.pack_hl as Tot_pack_hl,
                                                  , (sum(col("a.qtd_volume")) * sum(col("b.val_pack_hl"))).alias("tot_pack_hl")
                      # Join_Estudo_Base	Raw_Materials,	sum(o.Volume)* r.raw_hl as Tot_raw_hl,
                                                  , (sum(col("a.qtd_volume")) * sum(col("b.val_raw_hl"))).alias("tot_raw_hl")
                      # Join_Estudo_Base	Other_Variable_Expenses,	sum(o.Volume)* r.others_hl as Tot_others_hl,
                                                  , (sum(col("a.qtd_volume")) * sum(col("b.val_others_hl"))).alias("tot_others_hl")
                      # Join_Estudo_Base	Variable_COGS,	sum(o.Volume)* r.var_COGS_hl as Tot_var_COGS_hl,
                                                  , (sum(col("a.qtd_volume")) * sum(col("b.val_cogs_hl"))).alias("tot_var_cogs_hl")
                      # Join_Estudo_Base	Transfer_Freight,	sum(o.Volume)* r.Transfer_hl as Tot_Transfer_hl,
                                                  , (sum(col("a.qtd_volume")) * sum(col("b.val_transfer_hl"))).alias("tot_transfer_hl")
                      # Join_Estudo_Base	CIF_Freight,	sum(o.Volume)* r.CIF_Freight_hl as Tot_cif_hl,
                                                  , (sum(col("a.qtd_volume")) * sum(col("b.val_freight_hl"))).alias("tot_cif_hl")
                      # Join_Estudo_Base	Cif_Others,	sum(o.Volume)* r.CIF_Freight_Others_hl as Tot_CIF_Freight_Others_hl,
                                                  , (sum(col("a.qtd_volume")) * sum(col("b.val_cif_others_hl"))).alias("tot_cif_freight_others_hl")
                      # Join_Estudo_Base	Primary_Variable,	sum(o.Volume)* r.Prim_var_hl as Tot_transp_hl,
                                                  , (sum(col("a.qtd_volume")) * sum(col("b.val_prim_var_hl"))).alias("tot_transp_hl")
                                                )\
                                            .select(
                                                      # col("*")
                                                      col("id_material") 
                                                      , col("dsc_fbrc_vnd") 
                                                      , col("dsc_fbrc_prd") 
                                                      , col("id_uf") 
                                                      , col("dsc_cnl") 
                                                      , col("dt_mes") 
                                                      , col("dsc_scenario")
                                                      , col("id_key")
                                                      , col("tot_volume") 
                                                      , col("tot_fgp") 
                                                      , col("tot_gross_revenue") 
                                                      , col("tot_duties_hl") 
                                                      , col("tot_incent_hl") 
                                                      , col("tot_disc_hl") 
                                                      , col("tot_var_sel_exp_hl") 
                                                      , col("tot_proceeds_hl") 
                                                      , col("tot_rev_hl") 
                                                      , col("tot_resale_hl") 
                                                      , col("tot_energy_hl") 
                                                      , col("tot_pack_hl") 
                                                      , col("tot_raw_hl") 
                                                      , col("tot_others_hl") 
                                                      , col("tot_var_cogs_hl") 
                                                      , col("tot_transfer_hl") 
                                                      , col("tot_cif_hl") 
                                                      , col("tot_cif_freight_others_hl") 
                                                      , col("tot_transp_hl")
                                                      , col("dsc_versao")
                                                    )\
                                            # .display()

except Exception as e:
  print("Erro - ", var_etapa, f" Erro : {e}")

####
<b>Description:</b> Identificação dos scenarios Minimos e máximos<br>
<b>Regras:</b><br>
- min(col("dsc_scenario")).alias("dsc_scenario_min")
- max(col("dsc_scenario")).alias("dsc_scenario_max")


In [ ]:
try:
# if 1 == 1:
# Identificação dos cenarios minimo e máximo para calculos
  var_etapa = "11 - Identificando os valores de VPM baseados no cenário minimo e máximo do estudo informado"
  df_saida_vpm_min = df_saida_base_estudo_fim.alias("a")\
                                                .join(
                                                      df_saida_base_estudo_fim.groupBy()\
                                                                              .agg(
                                                                                    min(col("dsc_scenario")).alias("dsc_scenario_min")
                                                                                  )\
                                                                              .alias("min"), (col("a.dsc_scenario") == col("min.dsc_scenario_min")), "inner")\
                                                .alias("b")\
                                                .select(
                                                          col("b.id_material").alias("id_material")
                                                          , col("b.id_uf").alias("id_uf")
                                                          , col("b.dsc_cnl").alias("dsc_cnl")
                                                          , col("b.dt_mes").alias("dt_mes")
                                                          , col("dsc_fbrc_prd").alias("dsc_fbrc_prd")
                                                          , col("dsc_fbrc_vnd").alias("dsc_fbrc_vnd")
                                                          , col("b.dsc_scenario").alias("dsc_scenario")
                                                          , col("b.dsc_scenario_min").alias("dsc_scenario_min")
                                                          , coalesce(col("b.tot_volume"), lit(0)).alias("tot_volume")
                                                          , coalesce(col("b.tot_fgp"), lit(0)).alias("tot_fgp")
                                                        )\

  df_saida_vpm_max = df_saida_base_estudo_fim.alias("a")\
                                                .join(
                                                      df_saida_base_estudo_fim.groupBy()\
                                                                              .agg(
                                                                                    max(col("dsc_scenario")).alias("dsc_scenario_max")
                                                                                  )\
                                                                              .alias("max"), (col("a.dsc_scenario") == col("max.dsc_scenario_max")), "inner")\
                                                .alias("b")\
                                                .select(
                                                          col("b.id_material").alias("id_material")
                                                          , col("b.id_uf").alias("id_uf")
                                                          , col("b.dsc_cnl").alias("dsc_cnl")
                                                          , col("b.dt_mes").alias("dt_mes")
                                                          , col("dsc_fbrc_prd").alias("dsc_fbrc_prd")
                                                          , col("dsc_fbrc_vnd").alias("dsc_fbrc_vnd")
                                                          , col("b.dsc_scenario").alias("dsc_scenario")
                                                          , col("b.dsc_scenario_max").alias("dsc_scenario_max")
                                                          , coalesce(col("b.tot_volume"), lit(0)).alias("tot_volume")
                                                          , coalesce(col("b.tot_fgp"), lit(0)).alias("tot_fgp")
                                                        )\

  df_saida_vpm_minmax = df_saida_base_estudo_fim.alias("a")\
                                                .join(df_saida_vpm_min.alias("b"), (
                                                                                      (col("a.id_material") == col("b.id_material"))
                                                                                      & (col("a.id_uf") == col("b.id_uf"))
                                                                                      & (col("a.dsc_cnl") == col("b.dsc_cnl"))
                                                                                      & (col("a.dt_mes") == col("b.dt_mes"))
                                                                                      & (col("a.dsc_fbrc_prd") == col("b.dsc_fbrc_prd"))
                                                                                      & (col("a.dsc_fbrc_vnd") == col("b.dsc_fbrc_vnd"))
                                                                                      & (col("a.dsc_scenario") == col("b.dsc_scenario"))
                                                                                    )
                                                                                  , "left"
                                                      )\
                                                .join(df_saida_vpm_max.alias("c"), (
                                                                                      (col("a.id_material") == col("c.id_material"))
                                                                                      & (col("a.id_uf") == col("c.id_uf"))
                                                                                      & (col("a.dsc_cnl") == col("c.dsc_cnl"))
                                                                                      & (col("a.dt_mes") == col("c.dt_mes"))
                                                                                      & (col("a.dsc_fbrc_prd") == col("c.dsc_fbrc_prd"))
                                                                                      & (col("a.dsc_fbrc_vnd") == col("c.dsc_fbrc_vnd"))
                                                                                      & (col("a.dsc_scenario") == col("c.dsc_scenario"))
                                                                                    )
                                                                                  , "left"
                                                      )\
                                                .join(df_hierarq_produto.alias("d"), (col("a.id_material") == col("d.id_material")), "left")\
                                                .join(df_submarca.alias("e"), (col("d.id_submarca") == col("e.id_submarca")), "left")\
                                                .join(df_pacote.alias("f"), (col("d.id_pacote") == col("f.id_pacote")), "left")\
                                                .join(df_hierarq_produto_aux.alias("g"), (col("a.id_material") == col("g.id_material")), "left")\
                                        .select(
                                                  col("a.id_material").alias("id_material")
                                                  , col("a.id_uf").alias("id_uf")
                                                  , col("a.dsc_cnl").alias("dsc_cnl")
                                                  , col("a.dt_mes").alias("dt_mes")
                                                  , col("a.dsc_scenario").alias("dsc_scenario")
                                                  , col("d.id_submarca").alias("id_submarca")
                                                  # , col("e.dsc_submarca").alias("dsc_marca")
                                                  , coalesce(col("e.dsc_submarca"), col("g.dsc_marca")).alias("dsc_marca")
                                                  , col("d.id_pacote").alias("id_pacote")
                                                  # , col("f.dsc_pacote").alias("dsc_embalagem")
                                                  , coalesce(col("f.dsc_pacote"), col("g.dsc_embalagem")).alias("dsc_embalagem")
                                                  , when(col("a.dsc_scenario") == col("b.dsc_scenario"), col("b.tot_volume")).alias("tot_volume_min")
                                                  , when(col("a.dsc_scenario") == col("c.dsc_scenario"), col("c.tot_volume")).alias("tot_volume_max")
                                                  , when(col("a.dsc_scenario") == col("b.dsc_scenario"), col("b.tot_fgp")).alias("tot_fgp_min")
                                                  , when(col("a.dsc_scenario") == col("c.dsc_scenario"), col("c.tot_fgp")).alias("tot_fgp_max")
                                                )\

except Exception as e:
  print("Erro - ", var_etapa, f" Erro : {e}")

## Versionamento e Geração Final
<b>Description:</b> Controle de Versão para cada estudo realizado<br>
<b>Regras:</b><br>
- Será inserido um campo na tabela com a identificação da versão do estudo "20251118_160445"
- Mesmo que não haja nenhuma alteração de qualquer valor da carga, será gerada uma nova identificação de versão
- A chave para versionamento será a data em formato YYYYMMDD_HHMMSSS, informado via parâmetro pelo Pipeline
- Após a geração do cálculo, será disponibilizado o estudo, via sharepoint, gerando a saída ".xlsx" com a mesma identificação "_20251118_160445"

<b>Description:</b> Atualização final da Tabela a ser disponibilizada<br>
<b>Regras:</b><br>
- Gravação dos dados referente ao estudo<br>



In [ ]:
try:
# if 1 == 1:
# Selecionando os dados finais para gravação na tabela
  var_etapa = "6 - Geração das Tabelas Finais"
  if (val_lnh == 0):
    var_schema = "overwriteSchema"

    df_estudo_sop_fim = df_estudo_sop_fim.select(
                                                    col("id_material")
                                                    , col("dsc_fbrc_vnd")
                                                    , col("dsc_fbrc_prd")
                                                    , col("id_uf")
                                                    , col("dsc_cnl")
                                                    , col("dt_mes")
                                                    , col("dsc_centro_intrmd")
                                                    , col("dsc_scenario")
                                                    , col("qtd_volume")
                                                    , col("dsc_versao")
                                                  )

    df_estudo_sop_fim.write \
                  .format("delta")\
                  .mode(f"{var_modeatualiza}")\
                  .option(f"{var_schema}", "true")\
                  .option("delta.logRetentionDuration", "interval 30 days")\
                  .option("delta.deletedFileRetentionDuration", "interval 30 days")\
                  .option("comment", f"{tb_target_coment}")\
                  .saveAsTable(f"{silver}.{tb_target}")
                  # .option("path", f"/mnt/dls/30_dtmart/{tb_target}")\

  # print ("Carga = ", df_estudo_sop_carga.count())
  # print ("Carga Unica = ", df_estudo_sop_carga.distinct().count())
  # print ("Total = ", df_estudo_sop_fim.count())
  # print ("Erro Chaves = ", var_estudo_sopx_chave)
  # print ("Erro Datas = ", var_estudo_sopx_data)
  # print ("Erro Valores = ", var_estudo_sopx_valores)
  # print ("Erro UFs = ", var_uf)
  # print ("Erro Fabrica Venda = ", var_fabrica_vnd_dp)
  # print ("Erro Fabrica Prd = ", var_fabrica_prd_dp)
  # print ("Erro Canal = ", var_canal_dp)
  # print ("Parametro PL/NB = ", val_dtexec_ntbk)
  # print ("Val_Linhas = ", val_lnh)

  del df_estudo_sop_carga, df_estudo_sop_fim, df_canal_dp, df_estados, df_fabrica_dp 
  
  # Retornando a quantidade de linhas inconsistentes
  dbutils.notebook.exit(val_lnh)

except Exception as e:
  print("Erro - ", var_etapa, f" Erro : {e}")
